# Importing Lib.

In [26]:
import warnings
warnings.filterwarnings('ignore')

import os

import holidays
import datetime

import numpy as np
import pandas as pd
import polars as pl

import category_encoders as ce
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgbm
import catboost as cbt
import xgboost as xgb


# Classes and Defs

In [2]:
class DataLoader:
    root= 'D:\Vs_Code\enefit'
    lc_cl = ['longitude', 'latitude', 'county']
    #Weather Station Cordinates (lc)

    tg_cl = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 
             'datetime']
    #Target Columns Name (tg) 
    
    tr_cl = ['target', 'county', 'is_business', 'product_type', 'is_consumption',
             'datetime' ,'row_id']
    #Train Columns Name (tr)

    cl_cl =  ['product_type', 'county', 'eic_count', 'installed_capacity',
              'is_business','date']
    #Client Columns Name (cl)

    gs_cl = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
    #Gas Prices Columns Name (gs)

    ec_cl = ['forecast_date', 'euros_per_mwh']
    #Electric Price Columns Name (ec)

    fr_cl = ['latitude', 'longitude', 'hours_ahead', 'temperature',
             'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid',
             'cloudcover_total', '10_metre_u_wind_component',
             '10_metre_v_wind_component', 'forecast_datetime',
             'direct_solar_radiation', 'surface_solar_radiation_downwards',
             'snowfall', 'total_precipitation']
    #Forecast Weather Columns Name (fr)

    hs_cl = ['datetime','temperature','dewpoint','rain','snowfall',
             'surface_pressure','cloudcover_total','cloudcover_low',
             'cloudcover_mid','cloudcover_high','windspeed_10m',
             'winddirection_10m','shortwave_radiation','direct_solar_radiation',
             'diffuse_radiation','latitude','longitude']
    #Historical Weather Columns Name (hs)

    
    def __init__(self):

        self.df_tr=pl.read_csv(os.path.join(self.root, 'train.csv'),
                               columns=self.tr_cl, try_parse_dates=True)

        self.df_cl=pl.read_csv(os.path.join(self.root, 'client.csv')
                               ,columns=self.cl_cl, try_parse_dates=True)

        self.df_gs=pl.read_csv(os.path.join(self.root, 'gas_prices.csv'),
                               columns=self.gs_cl ,try_parse_dates=True)

        self.df_ec=pl.read_csv(os.path.join(self.root, 'electricity_prices.csv'),
                               columns=self.ec_cl, try_parse_dates=True)

        self.df_fr=pl.read_csv(os.path.join(self.root, 'forecast_weather.csv'),
                               columns=self.fr_cl,try_parse_dates=True)
                               
        self.df_hs=pl.read_csv(os.path.join(self.root, 'historical_weather.csv'),
                               columns=self.hs_cl,try_parse_dates=True)
        
        self.df_lc=pl.read_csv(os.path.join(self.root, 'weather_station_to_county_mapping.csv'),
                               columns=self.lc_cl)
        
        self.df_tr = self.df_tr.filter(
            pl.col('datetime') >= pd.to_datetime('2022-01-01'))
        
        self.df_tg=self.df_tr.select(self.tg_cl)

        self.schema_df_tr = self.df_tr.schema
        self.schema_df_cl = self.df_cl.schema
        self.schema_df_gs = self.df_gs.schema
        self.schema_df_ec = self.df_ec.schema
        self.schema_df_fr = self.df_fr.schema
        self.schema_df_hs = self.df_hs.schema
        self.schema_df_tg = self.df_tg.schema
        
        self.df_fr=self.df_fr.with_columns(
            pl.col('forecast_datetime').cast(pl.Datetime)
            )          
        
        self.df_lc = (self.df_lc.with_columns(
            pl.col('latitude').cast(pl.datatypes.Float32),
            pl.col('longitude').cast(pl.datatypes.Float32)
            )
        )
        
    def load_test_data(self,
                            df_ts_cl,
                            df_ts_gs,
                            df_ts_ec,
                            df_ts_fr,
                            df_ts_hs,
                            df_ts_tg
                            ):
       
        df_ts_cl=pl.from_pandas(
            df_ts_cl[self.df_cl], schema_overrides = self.schema_df_cl
        )

        df_ts_gs=pl.from_pandas(
            df_ts_gs[self.df_gs], schema_overrides = self.schema_df_gs
        )

        df_ts_ec=pl.from_pandas(
            df_ts_ec[self.df_ec], schema_overrides = self.schema_df_ec
        )

        df_ts_fr=pl.from_pandas(
            df_ts_fr[self.df_fr], schema_overrides = self.schema_df_fr
        )

        df_ts_hs=pl.from_pandas(
            df_ts_hs[self.df_hs], schema_overrides = self.schema_df_hs
        )

        df_ts_tg=pl.from_pandas(
            df_ts_tg[self.tg_cl], schema_overrides = self.schema_df_tg
        )


        self.df_cl=pl.concat([self.df_cl,df_ts_cl]).unique(
            ['date','county','is_bussiness','product_type']
            )
            
        self.df_gs=pl.concat([self.df_gs,df_ts_gs]).unique(
            ['forecast_date']
            )
            
        self.df_ec=pl.concat([self.df_ec,df_ts_ec]).unique(
            ['forecast_date']
            )
            
        self.df_fr=pl.concat([self.df_fr,df_ts_fr]).unique(
            ['forecast_datetime','latitude','longitude','hours_ahead']
            )
            
        self.df_hs=pl.concat([self.df_hs,df_ts_hs]).unique(
            ['datetime','latitude','longitude']
            )
            
        self.df_tg=pl.concat([self.df_tg,df_ts_tg]).unique(
            ['datetime', 'county', 'is_business', 'product_type', 'is_consumption']
            )
                
    def test_load(self,df_ts):
        df_ts=df_ts.reaname(columns={'prediction_datetime':'datetime'})
        df_ts=pl.from_pandas(df_ts[self.tr_cl[1:]], schema_overrides=self.schema_df_tr)

        return df_ts


In [3]:
class DataPreprocessor:
    
    def __init__(self,data):
        self.data=data

    def _add_date_feature(self, df_features):
        df_features=(
            df_features.with_columns(
                pl.col('datetime').dt.hour().alias('hour'),
                pl.col('datetime').dt.ordinal_day().alias('dayofyear'),
                pl.col('datetime').dt.weekday().alias('dayofweek'),
                pl.col('datetime').dt.day().alias('day'),
                pl.col('datetime').dt.month().alias('month'),
                pl.col('datetime').dt.year().alias('year')
            )
            .with_columns(
                pl.concat_str(
                    'county', 'is_business', 'product_type',
                    'is_consumption', separator='-'
                ).alias('segment')
            )
            .with_columns(
                (np.pi * pl.col('dayofyear') / 183).sin().alias('SinDayofyear'),
                (np.pi * pl.col('dayofyear') / 183).cos().alias('CosDayofyear'),
                (np.pi * pl.col('hour') / 12).sin().alias('SinHour'),
                (np.pi * pl.col('hour') / 12).cos().alias('CosHour')
            )
        )
        return df_features
    
    def _add_client_features(self, df_features):
        df_cl=self.data.df_cl

        df_features=df_features.join(
            df_cl.with_columns(
                (pl.col('date') + pl.duration(days=2)).cast(pl.Date)
            ),
            how='left', on=['county', 'is_business', 'product_type', 'date']
        )
        return df_features
    
    def _add_forecast_weather_features(self,df_features):
        df_fr=self.data.df_fr
        df_lc=self.data.df_lc

        df_fr = (
            df_fr.rename({'forecast_datetime': 'datetime'})
            .filter((pl.col('hours_ahead') >= 22) & (pl.col('hours_ahead') <= 45))
                    .drop('hours_ahead')
                    .with_columns(
                        pl.col('latitude').cast(pl.datatypes.Float32),
                        pl.col('longitude').cast(pl.datatypes.Float32)
                    )
                    .join(
                        df_lc, how='left', on=['longitude', 'latitude']
                    )
                    .drop('longitude', 'latitude')
        )
                        
        df_fr_date=(
            df_fr.groupby('datetime').mean().drop('county')
        )

        df_fr_local=(
            df_fr.filter(pl.col('county').is_not_null())
            .groupby('county','datetime')
            .mean()
        )

        for hours_lag in [0, 7 * 24]:
            df_features = df_features.join(
                df_fr_date.with_columns(
                    pl.col('datetime') + pl.duration(hours=hours_lag)
                ),
                on='datetime',
                how='left',
                suffix=f'forecast_{hours_lag}h',
            )
            df_features = df_features.join(
                df_fr_local.with_columns(
                    pl.col('datetime') + pl.duration(hours=hours_lag)
                ),
                on=['county', 'datetime'],
                how='left',
                suffix=f'forecast_local_{hours_lag}h',
            )

        return df_features
    
    def _add_historical_weather_features(self, df_features):
        df_hs = self.data.df_hs
        df_lc = self.data.df_lc

        df_hs = (
            df_hs.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_lc,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_hs_date = (
            df_hs.group_by("datetime").mean().drop("county")
        )

        df_hs_local = (
            df_hs.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )

        for hours_lag in [2 * 24, 7 * 24]:
            df_features = df_features.join(
                df_hs_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"historical_{hours_lag}h",
            )
            df_features = df_features.join(
                df_hs_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"historical_local_{hours_lag}h",
            )

        for hours_lag in [1 * 24]:
            df_features = df_features.join(
                df_hs_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag),
                    pl.col("datetime").dt.hour().alias("hour"),
                )
                .filter(pl.col("hour") <= 10)
                .drop("hour"),
                on="datetime",
                how="left",
                suffix=f"_historical_{hours_lag}h",
            )

        return df_features

    def _add_target_features(self,df_features):
        df_tg=self.data.df_tg

        df_tg_sum=(
            df_tg.groupby(['datetime', 'county', 'is_business', 'is_consumption'])
            .sum()
            .drop('product_type')
        )
        
        df_tg_county_sum=(
            df_tg.groupby(['datetime', 'is_business', 'is_consumption'])
            .sum()
            .drop('product_type','county')
        )

        for hours_lag in [
            2 * 24,
            3 * 24,
            4 * 24,
            5 * 24,
            6 * 24,
            7 * 24,
            8 * 24,
            9 * 24,
            10 * 24,
            11 * 24,
            12 * 24,
            13 * 24,
            14 * 24]:

            df_features=df_features.join(
                df_tg.with_columns(
                        pl.col('datetime')+pl.duration(hours=hours_lag)
                    )
                    .rename({'target': f'target_{hours_lag}h'}),
                how='left',
                on=['county','is_business','product_type',
                        'is_consumption','datetime']
            )
            
        for hours_lag in [2 * 24, 3 * 24, 7 * 24, 14 * 24]:
            df_features = df_features.join(
                df_tg_sum.with_columns(
                    pl.col('datetime') + pl.duration(hours=hours_lag)
                    )
                    .rename({'target': f'tg_sum_{hours_lag}h'}),
                on=['county', 'is_business', 'is_consumption', 'datetime'],
                how='left',
            )

            df_features = df_features.join(
                df_tg_county_sum.with_columns(
                    pl.col('datetime') + pl.duration(hours=hours_lag)
                )
                .rename({'target': f'tg_county_sum_{hours_lag}h'}),
                on=['is_business', 'is_consumption', 'datetime'],
                how='left',
                suffix=f'tg_county_sum_{hours_lag}h',
            )

        cols_for_stats = [
            f'target_{hours_lag}h' for hours_lag in [2 * 24, 3 * 24, 4 * 24, 5 * 24]
        ]

        df_features = df_features.with_columns(
            df_features.select(cols_for_stats).mean(axis=1).alias(f'target_mean'),
            df_features.select(cols_for_stats)
            .transpose()
            .std()
            .transpose()
            .to_series()
            .alias(f'target_std'),
        )

        for target_prefix, lag_nominator, lag_denomonator in [
            ('target', 24 * 7, 24 * 14),
            ('target', 24 * 2, 24 * 9),
            ('target', 24 * 3, 24 * 10),
            ('target', 24 * 2, 24 * 3),
            ('tg_sum', 24 * 2, 24 * 3),
            ('tg_sum', 24 * 7, 24 * 14),
            ('tg_county_sum', 24 * 2, 24 * 3),
            ('tg_county_sum', 24 * 7, 24 * 14),
        ]:
            df_features = df_features.with_columns(
                (
                    pl.col(f'{target_prefix}_{lag_nominator}h')
                    / (pl.col(f'{target_prefix}_{lag_denomonator}h') + 1e-3)
                ).alias(f'{target_prefix}_ratio_{lag_nominator}_{lag_denomonator}')
            )

        return df_features
    
    def _add_addinional_feature(self,df_features):
        
        df_features.with_columns(pl.col('is_business')
                                 .mul(pl.col('eic_count'))
                                 .mul(pl.col('installed_capacity'))
                                 .alias('best_feature_combo')
                                 )

        return df_features
    
    def _reduce_memory_usage(self, df_features):
        df_features = df_features.with_columns(pl.col(pl.Float64).cast(pl.Float32))
        return df_features

    def _drop_columns(self, df_features):
        df_features = df_features.drop(
            'date', 'datetime', 'hour', 'dayofyear'
        )
        return df_features

    def _to_pandas(self, df_features, y):
        cat_cols = ['county','is_business','product_type','is_consumption',
                    'segment',
        ]
 
        if y is not None:
            df_features = pd.concat([df_features.to_pandas(), y.to_pandas()], axis=1)
        else:
            df_features = df_features.to_pandas()

        df_features = df_features.set_index('row_id')
        df_features[cat_cols] = df_features[cat_cols].astype('category')

        return df_features,cat_cols

    def generate_features(self, df_prediction_items):
        if 'target' in df_prediction_items.columns:
            df_prediction_items, y = (
                df_prediction_items.drop('target'),
                df_prediction_items.select('target'),
            )
        else:
            y = None

        df_features = df_prediction_items.with_columns(
            pl.col('datetime').cast(pl.Date).alias('date'),
        )

        for add_features in [
            self._add_date_feature,
            self._add_client_features,
            self._add_forecast_weather_features,
            self._add_historical_weather_features,
            self._add_target_features,
            self._add_addinional_feature,
            self._reduce_memory_usage,
            self._drop_columns,
        ]:
            df_features = add_features(df_features)

        df_features,cat_cols = self._to_pandas(df_features, y)

        return df_features, cat_cols

In [28]:
class Modeler:
    
    def __init__(self,data,models,cat_cols):
        self.data=data
        self.models=models
        self.cat_cols=cat_cols

    def fit_and_pred(self):
        data=self.data
        cat_cols=self.cat_cols

        data[cat_cols[:-1]]=data[cat_cols[:-1]].astype('int16')
        value_counts=data[cat_cols[-1]].value_counts().to_dict()
        data[cat_cols[-1]]=data[cat_cols[-1]].map(value_counts)
        
        df0=data[data['is_business']==0]
        df1=data[data['is_business']==1]
        X0,X1,y0,y1=df0.drop('target', axis=1), df1.drop('target', axis=1), df0['target'], df1['target']


        tmp={'Model':[], 'Dilemma':[] ,'Scores':[], 'Std':[]}
        scores=[]
        models=self.models
        for model in(models):
            for dilemma in range(2):       
                if dilemma==0:  X,y=X0,y0 
                if dilemma==1:  X,y=X1,y1
                tmp['Model'].append(model.__class__.__name__)
                tmp['Dilemma'].append(dilemma)
                kf = KFold(n_splits=5, shuffle=True, random_state=42)
                for train_index, test_index in kf.split(X):
                    x_tr, x_ts = X.iloc[train_index], X.iloc[test_index]
                    y_tr, y_ts = y.iloc[train_index], y.iloc[test_index]
                
                    model.fit(x_tr,y_tr)
                    y_pr=model.predict(x_ts)
                    score=mean_squared_error(y_ts,y_pr,squared=False)
                    scores.append(score)
                tmp['Scores'].append(np.round(np.mean(scores),3))
                tmp['Std'].append(np.round(np.std(scores),3))
        
        return tmp
    
    def reporting(self,tmp):
        result=pd.DataFrame(tmp)
        return result
    
    def run(self):
        tmp = self.fit_and_pred()
        result =self.reporting(tmp)
        return result

# Initialisation

In [5]:
data=DataLoader()
enhance_data=DataPreprocessor(data)

# Make the data fell good

In [6]:
df_tr_features,cat_cols = enhance_data.generate_features(data.df_tr)
df_tr_features = df_tr_features[df_tr_features['target'].notnull()]

In [7]:
estonian_holidays = holidays.country_holidays('EE', years=range(2021, 2026))
estonian_holidays = list(estonian_holidays.keys())

def add_holidays_as_binary_features(df):
    df['country_holiday'] = df.apply(lambda row: (datetime.date(row['year'], row['month'], row['day']) in estonian_holidays) * 1, axis=1)
    
    return df

df_tr_features = add_holidays_as_binary_features(df_tr_features)

# Base Model Succsess

In [30]:
models=[xgb.XGBRegressor(),cbt.CatBoostRegressor(verbose=0),lgbm.LGBMRegressor(verbose=-1)]
hope=Modeler(df_tr_features,models,cat_cols=cat_cols)
result=hope.run()
result

,Model,Dilemma,Scores,Std
0,XGBRegressor,0,71.221,1.545
1,XGBRegressor,1,101.514,30.337
2,CatBoostRegressor,0,88.794,30.621
3,CatBoostRegressor,1,96.369,29.597
4,LGBMRegressor,0,92.992,27.324
5,LGBMRegressor,1,103.844,34.814
